In [1]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
def get_params_to_update(net, feature_extract):
    params_to_update = net.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in net.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in net.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    return params_to_update

def make_alexnet(out_features, feature_extract = False):
    net = models.alexnet(pretrained=True)
    set_parameter_requires_grad(net, feature_extract)
    num_ftrs = net.classifier[6].in_features
    net.classifier[6] = nn.Linear(num_ftrs, out_features)
    params_to_update = get_params_to_update(net, feature_extract)
    optimizer = optim.Adam(params_to_update, lr=0.0001)
    return net, optimizer

In [2]:
import statistics
import torch.utils.data as td
import net_training
from birds_dataset import Birds270Dataset
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets, models
import pandas as pd

# Splits a dataset randomly into a train and test set. The size of test set is 80% of the whole dataset.
# Then it trains the network
def train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size):
    train_set_size = int(len(dataset)*0.8)
    test_set_size = len(dataset)-train_set_size
    train_dataset, test_dataset = td.random_split(dataset, [train_set_size, test_set_size])
    label_set = dataset.get_label_set()
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
    results = net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set,
                                 epochs=epochs, optimizer=optimizer, print_results=True)
    return results

# Creates a few networks and trains them using a random split of the dataset.
# The number of created networks is in the "repeat" argument
# It returns the final validation results for each network
def cross_validate_net(net_generator, dataset, repeat=5, epochs=20, batch_size=32):
    all_results = []
    for i in range(repeat):
        print(f"Training network {i+1} ...")
        net, optimizer = net_generator()
        results = train_net_random_dataset_split(net, dataset, epochs, optimizer=optimizer, batch_size=batch_size)
        all_results.append(results)
        net_training.print_final_results(results)
    return all_results
    
    
def print_validation_results(cross_validation_results):
    losses = [r["loss"] for r in cross_validation_results]
    accuracies = [r["accuracy"] for r in cross_validation_results]
    print("Losses: ", losses)
    print("Loss:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(losses), statistics.stdev(losses)))
    print("Accuracies: ", accuracies)
    print("Accuracy:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(accuracies), statistics.stdev(accuracies)))
    
def results_to_dataframe(cross_validation_results):
    normalized = pd.json_normalize(cross_validation_results)
    return normalized
    


In [3]:
dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", "BALD EAGLE", "BARN OWL", "EURASIAN MAGPIE", "FLAMINGO",
                  "MALLARD DUCK", "OSTRICH", "PEACOCK", "PELICAN", "TRUMPTER SWAN"]

transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform)

def net_generator():
    return make_alexnet(out_features=len(selected_birds), feature_extract = False)

results = cross_validate_net(net_generator, dataset, repeat=5, epochs=100, batch_size=64)
print_validation_results(results)

Training network 1 ...
Params to learn:
	 features.0.weight
	 features.0.bias
	 features.3.weight
	 features.3.bias
	 features.6.weight
	 features.6.bias
	 features.8.weight
	 features.8.bias
	 features.10.weight
	 features.10.bias
	 classifier.1.weight
	 classifier.1.bias
	 classifier.4.weight
	 classifier.4.bias
	 classifier.6.weight
	 classifier.6.bias
Epoch 0:
	train loss: 0.8959168075083869
	validation loss: 0.29549413878623754, validation accuracy: 89.38356164383562%
	Elapsed time: 0:00:24.222645
Epoch 1:
	train loss: 0.12283864980851066
	validation loss: 0.18380184187107299, validation accuracy: 93.15068493150685%
	Elapsed time: 0:00:23.630345
Epoch 2:
	train loss: 0.038871295389113866
	validation loss: 0.11618579148429714, validation accuracy: 96.57534246575342%
	Elapsed time: 0:00:25.108044
Epoch 3:
	train loss: 0.009241498642525992
	validation loss: 0.1159592255541723, validation accuracy: 95.8904109589041%
	Elapsed time: 0:00:28.932125
Epoch 4:
	train loss: 0.003619421637093

Epoch 18:
	train loss: 0.00022376720337577956
	validation loss: 0.11969293123238707, validation accuracy: 95.8904109589041%
	Elapsed time: 0:00:22.283489
Epoch 19:
	train loss: 8.51341348287678e-05
	validation loss: 0.11777843228758197, validation accuracy: 96.57534246575342%
	Elapsed time: 0:00:22.290003
Epoch 20:
	train loss: 0.00012097843295369795
	validation loss: 0.11322435961194234, validation accuracy: 96.91780821917808%
	Elapsed time: 0:00:22.579707
Epoch 21:
	train loss: 5.7265833930890585e-05
	validation loss: 0.11933713681893805, validation accuracy: 96.57534246575342%
	Elapsed time: 0:00:22.545902
Epoch 22:
	train loss: 0.00019750292454854564
	validation loss: 0.1307342710764441, validation accuracy: 96.91780821917808%
	Elapsed time: 0:00:22.477000
Epoch 23:
	train loss: 8.274104311567498e-05
	validation loss: 0.12084993223455569, validation accuracy: 96.91780821917808%
	Elapsed time: 0:00:22.518926
Epoch 24:
	train loss: 0.00011742654725828162
	validation loss: 0.150632284

Params to learn:
	 features.0.weight
	 features.0.bias
	 features.3.weight
	 features.3.bias
	 features.6.weight
	 features.6.bias
	 features.8.weight
	 features.8.bias
	 features.10.weight
	 features.10.bias
	 classifier.1.weight
	 classifier.1.bias
	 classifier.4.weight
	 classifier.4.bias
	 classifier.6.weight
	 classifier.6.bias
Epoch 0:
	train loss: 0.8544558687439096
	validation loss: 0.16990300359791272, validation accuracy: 94.52054794520548%
	Elapsed time: 0:00:21.561333
Epoch 1:
	train loss: 0.12373583473099034
	validation loss: 0.14084223019954276, validation accuracy: 95.54794520547945%
	Elapsed time: 0:00:21.738859
Epoch 2:
	train loss: 0.04655490989603524
	validation loss: 0.08106253204280384, validation accuracy: 97.6027397260274%
	Elapsed time: 0:00:21.595243
Epoch 3:
	train loss: 0.011400138615833446
	validation loss: 0.06718354006234097, validation accuracy: 97.94520547945206%
	Elapsed time: 0:00:21.669087
Epoch 4:
	train loss: 0.00627402119619267
	validation loss: 0.

Epoch 12:
	train loss: 0.00022087908968150514
	validation loss: 0.12277419034595767, validation accuracy: 97.26027397260275%
	Elapsed time: 0:00:21.097681
Epoch 13:
	train loss: 0.0002842309046768816
	validation loss: 0.11648837127403854, validation accuracy: 97.26027397260275%
	Elapsed time: 0:00:20.990485
Epoch 14:
	train loss: 0.0002472656526243737
	validation loss: 0.12291416791203903, validation accuracy: 97.26027397260275%
	Elapsed time: 0:00:21.015417
Epoch 15:
	train loss: 0.00015007047828104775
	validation loss: 0.13430099505675983, validation accuracy: 97.6027397260274%
	Elapsed time: 0:00:20.970555
Epoch 16:
	train loss: 0.0002207467296539357
	validation loss: 0.12581238354722116, validation accuracy: 97.6027397260274%
	Elapsed time: 0:00:21.032317
Epoch 17:
	train loss: 7.590916180408297e-05
	validation loss: 0.12248088480675057, validation accuracy: 97.6027397260274%
	Elapsed time: 0:00:21.021896
Epoch 18:
	train loss: 8.534838708667132e-05
	validation loss: 0.119192570575

In [4]:
dataframe = results_to_dataframe(results)
dataframe.to_csv("../results/alexnet_pretrained.csv")